# **Colab**

In [ ]:
# === 1. Montagem do Google Drive e definição dos caminhos ===
from google.colab import drive
drive.mount('/content/drive')
arquivo_tratado = '/content/drive/My Drive/Colab/cb-classificador/data/dados_tratados.csv'
arquivo_codificado = '/content/drive/My Drive/Colab/cb-classificador/data/dados_codificados.csv'
arquivo_vocabulario = '/content/drive/My Drive/Colab/cb-classificador/data/dados_vocabulario.csv'
arquivo_encoder = '/content/drive/My Drive/Colab/cb-classificador/data/onehot_encoder.pkl'
arquivo_vectorizer = '/content/drive/My Drive/Colab/cb-classificador/data/tfidf_vectorizer.pkl'

Mounted at /content/drive


In [ ]:
# === 2. Instalação e importação de bibliotecas ===
!pip install unidecode
from unidecode import unidecode
import pandas as pd
import joblib
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 4.6 MB/s eta 0:00:00


In [ ]:
# === 3. Função para codificar variáveis categóricas ===
def codificar_variaveis_categoricas(df, colunas_categoricas, caminho_encoder):
    encoder = OneHotEncoder(handle_unknown='ignore')
    X_categorico = encoder.fit_transform(df[colunas_categoricas])
    joblib.dump(encoder, caminho_encoder) # Salvar o encoder para uso no protótipo
    return X_categorico, encoder

In [ ]:
# === 4. Função para vetorizar variável textual ===
def vetorizar_texto(df, coluna_texto, caminho_vectorizer, caminho_vocabulario, min_df=10):
    vectorizer = TfidfVectorizer(min_df=min_df)
    X_texto = vectorizer.fit_transform(df[coluna_texto])
    joblib.dump(vectorizer, caminho_vectorizer) # Salvar o vectorizer para uso no protótipo
    df_vocab = pd.DataFrame(vectorizer.get_feature_names_out(), columns=['Termo'])
    df_vocab.to_csv(caminho_vocabulario, index=False) # Salvar o vocabulário
    return X_texto

In [ ]:
# === 5. Leitura dos dados ===
df = pd.read_csv(arquivo_tratado)

In [ ]:
# === 6. Verificação do balanceamento das classes da variável alvo ===
total = df['categoria'].count()
print(f"Percentual CB: {df['categoria'].value_counts().get('CB', 0)/total*100:.2f}%")
print(f"Percentual XX: {df['categoria'].value_counts().get('XX', 0)/total*100:.2f}%")

Percentual CB: 42.01%
Percentual XX: 57.99%


In [ ]:
len(df)

1914

In [ ]:
df['categoria'].value_counts()

,count
categoria,
XX,1110
CB,804


In [ ]:
df['cod_ncm'].value_counts()

,count
cod_ncm,
9,290
27101932,259
27101921,236
27101259,220
84212300,118
84219999,87
22011000,80
31021010,74
34029039,41


In [ ]:
df['cod_anp'].value_counts()

,count
cod_anp,
0,1363
820101034,106
620505001,88
320102001,76
9,74
320102002,74
820101033,43
810101001,32
740101006,15


In [ ]:
df['cod_unidade'].value_counts()

,count
cod_unidade,
un,1197
lt,615
pc,73
9,13
hr,10
gl,5
dl,1


In [ ]:
# === 7. Codificação das variáveis categóricas ===
colunas_cat = ['cod_ncm', 'cod_anp', 'cod_unidade']
X_categorico, encoder = codificar_variaveis_categoricas(df, colunas_cat, arquivo_encoder)

In [ ]:
# === 8. Vetorização da variável textual ===
df['txt_descricao'] = df['des_anp'].fillna('') + ' | ' + df['des_produto'].fillna('')
X_texto = vetorizar_texto(df, 'txt_descricao', arquivo_vectorizer, arquivo_vocabulario)

In [ ]:
# === 9. Combinação das variáveis preditoras codificadas e vetorizadas ===
X = pd.concat([
    pd.DataFrame(X_categorico.toarray(), columns=encoder.get_feature_names_out(['cod_ncm', 'cod_anp', 'cod_unidade'])),
    pd.DataFrame(X_texto.toarray())
], axis=1)
X.columns = X.columns.astype(str)

In [ ]:
# === 10. Variável alvo ===
y = df['categoria']

In [ ]:
# === 11. Salvamento do dataset final ===
df_codificado = pd.concat([X, y], axis=1)
df_codificado.to_csv(arquivo_codificado, index=False)

In [ ]:
len(df_codificado)

1914

In [ ]:
len(df_codificado.columns)

186